In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, year, to_timestamp, count, dense_rank, row_number
)
from pyspark.sql.window import Window
import pandas as pd

spark = SparkSession.builder.getOrCreate()

In [0]:
item_pd = pd.read_csv(
    "https://merkle-de-interview-case-study.s3.eu-central-1.amazonaws.com/de/item.csv",
    dtype=str
)

event_pd = pd.read_csv(
    "https://merkle-de-interview-case-study.s3.eu-central-1.amazonaws.com/de/event.csv",
    dtype=str
)

In [0]:
item_bronze_df = spark.createDataFrame(item_pd)
event_bronze_df = spark.createDataFrame(event_pd)

In [0]:
item_bronze_df.write.mode("overwrite").saveAsTable("bronze_item")
event_bronze_df.write.mode("overwrite").saveAsTable("bronze_event")

In [0]:
item_silver_df = (
    spark.table("bronze_item")
    .withColumn("id", col("id").try_cast("int"))
    .withColumn("name", col("name").cast("string"))
)

item_silver_df.write.mode("overwrite").saveAsTable("silver_item")

In [0]:
event_silver_df = (
    spark.table("bronze_event")
    .withColumn("platform", col("platform").cast("string"))
    .withColumn("event_time", to_timestamp(col("event_time")))
    .withColumn("event_year", year(col("event_time")))
)

event_silver_df.write \
    .mode("overwrite") \
    .partitionBy("event_year") \
    .saveAsTable("silver_event")



    event_id:string
event_time:string
user_id:string
event.payload:string